In [ ]:
# =============================================================================
# Phase 5.5 — Feature Extraction (EEG μ-band + richer EMG TD)
# -----------------------------------------------------------------------------
# =============================================================================

from __future__ import annotations

from pathlib import Path
from typing import List, Tuple, Dict

import numpy as np
from scipy.signal import welch

# ---------------- CONFIG ----------------

# ⚠️ Set this to the same ROOT_DIR used in Phase 5 / Phase 6 (ICML + BioRob).
ROOT_DIR = Path(r"/home/tsultan1/BioRob(Final)/Data")

DATASET_DIR = ROOT_DIR / "_dataset_icml_v1"

# Balanced supervised exports from Phase 5
EXPORT_PREFIX_BALANCED = "exports_v1_balanced"

# Feature file prefix (must match CFG.EEG_PSD_PREFIX in Phase 6)
FEATURE_PREFIX = "features_v1_eeg_psd_full"

# Splits to process
SPLITS = ["train", "val", "test"]

# Synchronized sampling rate and window length (must match Phase 4 / 5)
FS = 250.0          # Hz (assumed sync frequency)
WIN_LEN = 500       # samples per window

# EEG bands (Hz) — includes μ-band explicitly
EEG_BANDS: Dict[str, Tuple[float, float]] = {
    "delta": (1.0, 4.0),
    "theta": (4.0, 8.0),
    "mu":    (8.0, 13.0),   # μ / alpha-mu band (motor-related)
    "beta":  (13.0, 30.0),
}

# Welch params
WELCH_NPERSEG = 256

# Small epsilon for log / division stability
EPS = 1e-8

# ---------------- UTILITIES ----------------

def discover_folds(prefix: str) -> List[int]:
    folds = []
    for p in DATASET_DIR.glob(f"{prefix}_fold*"):
        name = p.name
        try:
            fid = int(name.split("fold")[-1])
            folds.append(fid)
        except ValueError:
            continue
    return sorted(set(folds))


def _safe_log(x: np.ndarray, eps: float = EPS) -> np.ndarray:
    return np.log(np.maximum(x, eps))


# ---------------- EEG FEATURES ----------------

def compute_eeg_features_window(x: np.ndarray, fs: float) -> np.ndarray:
    """
    x: (T, C_eeg)
    Returns feature vector (D_eeg,) per window:
      [log bandpower (4 bands) + relative bandpower (4 bands)
       + spectral entropy + Hjorth (activity/mobility/complexity)] for each channel.
    """
    T, C = x.shape
    if C == 0 or T == 0:
        return np.zeros((0,), dtype=np.float32)

    # Welch PSD: f (F,), Pxx (F, C)
    f, Pxx = welch(
        x,
        fs=fs,
        axis=0,
        nperseg=min(WELCH_NPERSEG, T),
        noverlap=None,
        detrend="constant",
    )

    # Limit to 1–30 Hz range for total power / entropy
    band_mask_1_30 = (f >= 1.0) & (f <= 30.0)
    Pxx_1_30 = Pxx[band_mask_1_30]  # (F_band, C)
    total_power = Pxx_1_30.sum(axis=0) + EPS  # (C,)

    # Absolute and relative bandpowers per band & channel
    abs_band_feats = []
    rel_band_feats = []

    for name, (lo, hi) in EEG_BANDS.items():
        mask = (f >= lo) & (f <= hi)
        if not np.any(mask):
            # Fallback if no freq bin hits band (shouldn't happen with 250 Hz and nperseg>=128)
            bp = np.zeros((C,), dtype=np.float32)
        else:
            bp = Pxx[mask].sum(axis=0)  # (C,)
        abs_band_feats.append(_safe_log(bp))          # log-absolute bandpower
        rel_band_feats.append(bp / total_power)       # relative bandpower vs 1–30 Hz

    abs_band = np.stack(abs_band_feats, axis=0)  # (B, C)
    rel_band = np.stack(rel_band_feats, axis=0)  # (B, C)

    # Spectral entropy (1–30 Hz)
    P_norm = Pxx_1_30 / (Pxx_1_30.sum(axis=0, keepdims=True) + EPS)
    spec_entropy = -(P_norm * _safe_log(P_norm)).sum(axis=0)  # (C,)

    # Hjorth parameters per channel
    # activity = var(x)
    # mobility = sqrt(var(dx)/var(x))
    # complexity = mobility(dx) / mobility(x)
    dx  = np.diff(x, axis=0)          # (T-1, C)
    ddx = np.diff(dx, axis=0)         # (T-2, C)

    var_x   = np.var(x,   axis=0) + EPS
    var_dx  = np.var(dx,  axis=0) + EPS
    var_ddx = np.var(ddx, axis=0) + EPS

    activity    = var_x
    mobility    = np.sqrt(var_dx / var_x)
    mobility_dx = np.sqrt(var_ddx / var_dx)
    complexity  = mobility_dx / (mobility + EPS)

    hjorth = np.stack([activity, mobility, complexity], axis=0)  # (3, C)
    hjorth = _safe_log(hjorth)  # log-scale Hjorth for stability

    # Stack all per-channel features:
    # shape: abs_band (4,C), rel_band (4,C), spec_entropy (1,C), hjorth (3,C)
    # total: (12, C) → flatten to (12*C,)
    spec_entropy_row = spec_entropy[None, :]  # (1, C)
    all_feats = np.concatenate([abs_band, rel_band, spec_entropy_row, hjorth], axis=0)  # (12, C)
    feats_flat = all_feats.reshape(-1)

    return feats_flat.astype(np.float32)


# ---------------- EMG FEATURES ----------------

def _zero_crossings(x: np.ndarray, thr: float = 0.0) -> int:
    """
    Zero-crossing count with optional threshold.
    x: (T,)
    """
    if x.size < 2:
        return 0
    x1 = x[:-1]
    x2 = x[1:]
    crosses = (x1 * x2) < 0
    if thr > 0.0:
        amp_ok = (np.abs(x1 - x2) >= thr) | (np.abs(x2 - x1) >= thr)
        crosses = crosses & amp_ok
    return int(crosses.sum())


def _ssc(x: np.ndarray, thr: float = 0.0) -> int:
    """
    Slope sign changes (SSC) with threshold.
    x: (T,)
    """
    if x.size < 3:
        return 0
    x1 = x[:-2]
    x2 = x[1:-1]
    x3 = x[2:]
    cond = ( (x2 - x1) * (x2 - x3) > 0 )  # change in slope
    if thr > 0.0:
        amp_ok = (np.abs(x2 - x1) >= thr) | (np.abs(x3 - x2) >= thr)
        cond = cond & amp_ok
    return int(cond.sum())


def compute_emg_features_window(x: np.ndarray) -> np.ndarray:
    """
    x: (T, C_emg)
    Returns feature vector (D_emg,) per window:
      [RMS, MAV, WL (log), ZC (log), SSC (log), log-variance] per channel.
    """
    T, C = x.shape
    if C == 0 or T == 0:
        return np.zeros((0,), dtype=np.float32)

    feats_ch = []
    for c in range(C):
        sig = x[:, c]

        # RMS
        rms = np.sqrt(np.mean(sig ** 2))

        # MAV (mean absolute value)
        mav = np.mean(np.abs(sig))

        # Waveform length (WL)
        wl = np.sum(np.abs(np.diff(sig)))

        # Zero-crossings (with tiny threshold to avoid noise)
        zc = _zero_crossings(sig, thr=0.01)

        # Slope sign changes (SSC)
        ssc = _ssc(sig, thr=0.01)

        # Variance (log)
        var_val = np.var(sig)

        # Compress scales for WL, ZC, SSC, var with log(1+x)
        wl_log  = np.log(1.0 + max(wl, 0.0))
        zc_log  = np.log(1.0 + max(float(zc), 0.0))
        ssc_log = np.log(1.0 + max(float(ssc), 0.0))
        var_log = np.log(1.0 + max(var_val, 0.0))

        feats_ch.append([rms, mav, wl_log, zc_log, ssc_log, var_log])

    feats = np.array(feats_ch, dtype=np.float32).reshape(-1)  # (C*6,)
    return feats.astype(np.float32)


# ---------------- MAIN EXTRACTION ----------------

def extract_features_for_split(
    fold_id: int,
    split: str,
) -> Tuple[np.ndarray, np.ndarray]:
    """
    For one fold and split:
      - Load all shard_*.npz from exports_v1_balanced_fold{fold}/{split}
      - Compute EEG + EMG features window-wise
      - Return (X_eeg_feats, X_emg_feats) with matching N
        (order follows the order of windows in shards).
    """
    fold_dir = DATASET_DIR / f"{EXPORT_PREFIX_BALANCED}_fold{fold_id}"
    split_dir = fold_dir / split
    if not split_dir.exists():
        raise FileNotFoundError(f"Split dir not found: {split_dir}")

    shard_paths = sorted(split_dir.glob("*_shard_*.npz"))
    if not shard_paths:
        raise FileNotFoundError(f"No shard npz files in {split_dir}")

    eeg_feats_list = []
    emg_feats_list = []
    total_windows = 0

    print(f"[Fold {fold_id}][{split}] Found {len(shard_paths)} shards")

    for sp in shard_paths:
        z = np.load(sp, allow_pickle=True)
        X_eeg = z["X_EEG"]   # (N, T, C_eeg)
        X_emg = z["X_EMG"]   # (N, T, C_emg)

        N, T_eeg, C_eeg = X_eeg.shape
        _, T_emg, C_emg = X_emg.shape

        # Sanity checks on time length
        if T_eeg != WIN_LEN:
            print(f"  [warn] EEG window len {T_eeg} != {WIN_LEN} in {sp.name}")
        if T_emg != WIN_LEN:
            print(f"  [warn] EMG window len {T_emg} != {WIN_LEN} in {sp.name}")

        # If you ever hit C_eeg==0 or C_emg==0 here, something is off with Phase 5.
        # You can assert or just let compute_* return zeros (0-dim, which would break stacking).
        # For your current pipeline we assume C_eeg>0 and C_emg>0 for all windows.
        if C_eeg == 0:
            raise RuntimeError(f"[Fold {fold_id}][{split}] X_EEG has 0 channels in {sp.name}")
        if C_emg == 0:
            raise RuntimeError(f"[Fold {fold_id}][{split}] X_EMG has 0 channels in {sp.name}")

        for i in range(N):
            eeg_win = X_eeg[i]  # (T, C_eeg)
            emg_win = X_emg[i]  # (T, C_emg)

            # Guard: ensure 2D with expected shape
            eeg_win = np.asarray(eeg_win, dtype=np.float32).reshape(T_eeg, C_eeg)
            emg_win = np.asarray(emg_win, dtype=np.float32).reshape(T_emg, C_emg)

            eeg_feat = compute_eeg_features_window(eeg_win, fs=FS)
            emg_feat = compute_emg_features_window(emg_win)

            eeg_feats_list.append(eeg_feat)
            emg_feats_list.append(emg_feat)
            total_windows += 1

    if total_windows == 0:
        raise RuntimeError(f"[Fold {fold_id}][{split}] No windows collected for features")

    X_eeg_feats = np.stack(eeg_feats_list, axis=0)  # (N, D_eeg)
    X_emg_feats = np.stack(emg_feats_list, axis=0)  # (N, D_emg)

    print(
        f"[Fold {fold_id}][{split}] Extracted EEG feats shape={X_eeg_feats.shape}, "
        f"EMG feats shape={X_emg_feats.shape}"
    )
    return X_eeg_feats.astype(np.float32), X_emg_feats.astype(np.float32)


def run_phase_5_5():
    print("========== Phase 5.5 — EEG μ-band + rich EMG feature extraction ==========")
    print(f"ROOT_DIR    : {ROOT_DIR}")
    print(f"DATASET_DIR : {DATASET_DIR}")
    print(f"Exports     : {EXPORT_PREFIX_BALANCED}_foldK")
    folds = discover_folds(EXPORT_PREFIX_BALANCED)
    if not folds:
        raise SystemExit(
            f"No balanced exports found with prefix {EXPORT_PREFIX_BALANCED}_fold* in {DATASET_DIR}"
        )
    print(f"Folds found : {folds}")

    for fid in folds:
        print(f"\n===================== FOLD {fid} =====================")
        for split in SPLITS:
            try:
                X_eeg_feats, X_emg_feats = extract_features_for_split(fid, split)
            except FileNotFoundError as e:
                print(f"[Fold {fid}][{split}] skip: {e}")
                continue

            out_path = DATASET_DIR / f"{FEATURE_PREFIX}_fold{fid}_{split}.npz"

            # Save as X_psd (EEG) + X_emg (EMG)
            np.savez_compressed(
                out_path,
                X_psd=X_eeg_feats,   # what Phase 6 expects for EEG_PSD_PREFIX
                X_emg=X_emg_feats,   # optional EMG features for future use
            )
            print(
                f"[Fold {fid}][{split}] Saved features → {out_path} "
                f"(X_psd: {X_eeg_feats.shape}, X_emg: {X_emg_feats.shape})"
            )

    print("\n🎉 Phase 5.5 complete!")
    print(f"Feature prefix: {FEATURE_PREFIX}_foldK_{'{train,val,test}'}")
    print("EEG features: abs+relative bandpowers (delta/theta/mu/beta), spectral entropy, Hjorth per channel")
    print("EMG features: RMS, MAV, log-WL, log-ZC, log-SSC, log-var per channel")


if __name__ == "__main__":
    run_phase_5_5()